In [12]:
import cv2 as cv
import numpy as np

In [13]:
def avg_frame(video_path, max_frames=300):
    """
    Calcula o fundo (background) de um vídeo com base na média de todos os frames de um vídeo.

    Args:
        video_path (str): Diretoria do ficheiro de vídeo.
        max_frames (int, o): Número máximo de frames a utilizar para a média. Por defeito, 300.
    Returns:
        np.ndarray: "Frame médio", formatado como um array numpy de inteiros positivos de 8 bits.
    """
    cap = cv.VideoCapture(video_path)
    count = 0
    avg = None

    while count < max_frames:
        ret, frame = cap.read()
        if not ret:
            break

        f = frame.astype(np.float32)
        if avg is None:
            avg = f
        else:
            avg += f

        count += 1

    cap.release()

    avg /= count
    return avg.astype(np.uint8)


In [14]:
# Example usage:
video_path = "resources/AutoEstrada.avi"

avg_img = avg_frame(video_path)

cv.imshow("Average Image", avg_img)
cv.waitKey(0)
cv.destroyAllWindows()


In [15]:
def detect_cars(video_path, output_path="output_cars.mp4"):
    """
    Detecta e marca veículos num vídeo usando subtração do fundo por média de frames.

    Args:
        video_path (str): Caminho para o ficheiro de vídeo de entrada.
        output_path (str, optional): Caminho para o ficheiro de vídeo de saída (mp4). Padrão: "output_cars.mp4".
    """
    background = avg_frame(video_path)

    cap = cv.VideoCapture(video_path)

    # Prepare video writer
    w = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv.CAP_PROP_FPS)

    fourcc = cv.VideoWriter_fourcc(*'mp4v')
    out = cv.VideoWriter(output_path, fourcc, fps, (w, h))

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        diff = cv.absdiff(frame, background)
        gray = cv.cvtColor(diff, cv.COLOR_BGR2GRAY)

        _, thresh = cv.threshold(gray, 30, 255, cv.THRESH_BINARY)

        kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (5, 5))
        mask = cv.morphologyEx(thresh, cv.MORPH_CLOSE, kernel, iterations=2)
        mask = cv.morphologyEx(mask, cv.MORPH_OPEN, kernel, iterations=2)

        contours, _ = cv.findContours(mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

        for cnt in contours:
            if cv.contourArea(cnt) < 500:
                continue

            x, y, w2, h2 = cv.boundingRect(cnt)
            cv.rectangle(frame, (x, y), (x + w2, y + h2), (0, 255, 0), 2)

        out.write(frame)
        cv.imshow("Car Detection", frame)

        if cv.waitKey(1) & 0xFF == 27:
            break

    cap.release()
    out.release()
    cv.destroyAllWindows()
    print("Done:", output_path)


In [16]:
# Example usage:
video_path = "resources/AutoEstrada.avi"

# -------- RUN THE DETECTOR --------
detect_cars(video_path, "cars_detected.mp4")

Done: cars_detected.mp4


In [ ]:
def detect_cars_polygon(video_path, output_path, roi_polygon):
    """
    Detecta e marca veículos dentro de uma região de interesse (ROI) poligonal num vídeo.

    Args:
        video_path (str): Caminho para o ficheiro de vídeo de entrada.
        output_path (str): Caminho para o ficheiro de vídeo de saída (mp4).
        roi_polygon (np.ndarray): Array Nx2 (int32) com os vértices do polígono da ROI, ex. np.array([[x1,y1], [x2,y2], ...], dtype=np.int32).
    """
    background = avg_frame(video_path)
    cap = cv.VideoCapture(video_path)

    w = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv.CAP_PROP_FPS)

    # Create polygon mask
    mask_roi = np.zeros((h, w), dtype=np.uint8)
    cv.fillPoly(mask_roi, [roi_polygon], 255)

    out = cv.VideoWriter(output_path,
                         cv.VideoWriter_fourcc(*"mp4v"),
                         fps, (w, h))

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Background subtraction
        diff = cv.absdiff(frame, background)
        gray = cv.cvtColor(diff, cv.COLOR_BGR2GRAY)

        # Apply polygon mask
        masked_gray = cv.bitwise_and(gray, gray, mask=mask_roi)

        # Threshold
        _, thresh = cv.threshold(masked_gray, 30, 255, cv.THRESH_BINARY)

        # Morphology
        kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (5, 5))
        cleaned = cv.morphologyEx(thresh, cv.MORPH_CLOSE, kernel, iterations=2)
        cleaned = cv.morphologyEx(cleaned, cv.MORPH_OPEN, kernel, iterations=2)

        # Find contours
        contours, _ = cv.findContours(cleaned, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

        for cnt in contours:
            if cv.contourArea(cnt) < 500:
                continue

            x, y, w2, h2 = cv.boundingRect(cnt)
            cv.rectangle(frame, (x, y), (x+w2, y+h2), (0, 255, 0), 2)

        # Draw polygon overlay
        cv.polylines(frame, [roi_polygon], True, (255, 0, 0), 2)

        out.write(frame)
        cv.imshow("Car Detection", frame)

        if cv.waitKey(1) & 0xFF == 27:
            break

    cap.release()
    out.release()
    cv.destroyAllWindows()
    print("Polygon ROI detection finished. Saved:", output_path)

In [18]:
# Example usage:
video_path = "resources/AutoEstrada.avi"
roi_polygon = np.array([
    [100, 70],   # top-left
    [155, 70],  # top-right
    [230, 240],  # bottom-right
    [5, 240]     # bottom-left
], dtype=np.int32)

# -------- RUN THE DETECTOR --------
detect_cars_polygon(video_path, "cars_detected.mp4", roi_polygon)

Polygon ROI detection finished. Saved: cars_detected.mp4
